# 2.2.3 色彩空間 (Color Spaces)

**WBS 2.2.3**: 色彩空間轉換與應用

本模組涵蓋 OpenCV 中的各種色彩空間，從基礎概念到實際應用。

## 學習目標
- 理解不同色彩空間的特性與用途
- 掌握 BGR、RGB、HSV、LAB、灰階之間的轉換
- 學習色彩空間在實際應用中的選擇策略
- 實作色彩檢測、色彩增強等應用

## 前置知識
- Python 基礎語法
- NumPy 陣列操作
- OpenCV 圖像讀取與顯示

In [ ]:
# 導入必要的庫
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# 設置中文顯示
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False

# 設置圖像顯示大小
plt.rcParams['figure.figsize'] = (12, 8)

## 1. 色彩空間基礎概念

### 什麼是色彩空間？

色彩空間（Color Space）是一種用數學方式描述顏色的模型。不同的色彩空間有不同的表示方法和應用場景。

### 常見色彩空間

1. **BGR/RGB** - 加法混色模型
   - 基於紅(R)、綠(G)、藍(B)三原色
   - OpenCV 預設使用 BGR 順序
   - 適合：圖像顯示、基本處理

2. **HSV** - 色相、飽和度、明度
   - H (Hue): 色相 0-180°
   - S (Saturation): 飽和度 0-255
   - V (Value): 明度 0-255
   - 適合：色彩檢測、物體追蹤

3. **LAB** - 感知均勻色彩空間
   - L: 亮度 (0-100)
   - a: 紅綠軸 (-128 to 127)
   - b: 藍黃軸 (-128 to 127)
   - 適合：色彩測量、色差計算

4. **Grayscale** - 灰階
   - 單通道，0-255
   - 適合：邊緣檢測、形態學操作

## 2. BGR 與 RGB 色彩空間

### OpenCV 使用 BGR 順序的原因

歷史原因：早期相機和掃描器製造商（如 Bayer）採用 BGR 順序。OpenCV 為保持兼容性，延續了這個傳統。

In [ ]:
# 載入測試圖像
img_path = '../assets/images/basic/lena.jpg'
if not Path(img_path).exists():
    # 如果 lena 不存在，創建一個彩色測試圖像
    img_bgr = np.zeros((300, 300, 3), dtype=np.uint8)
    img_bgr[:100, :, 0] = 255  # Blue
    img_bgr[100:200, :, 1] = 255  # Green
    img_bgr[200:, :, 2] = 255  # Red
else:
    img_bgr = cv2.imread(img_path)

# BGR to RGB 轉換
img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

# 顯示比較
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

axes[0].imshow(cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB))
axes[0].set_title('Original (BGR → RGB for display)')
axes[0].axis('off')

axes[1].imshow(img_rgb)
axes[1].set_title('Converted RGB')
axes[1].axis('off')

plt.tight_layout()
plt.show()

print(f"BGR shape: {img_bgr.shape}")
print(f"RGB shape: {img_rgb.shape}")

### 通道分離與合併

In [ ]:
# BGR 通道分離
b, g, r = cv2.split(img_bgr)

# 顯示各通道
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# 原圖
axes[0, 0].imshow(cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB))
axes[0, 0].set_title('Original Image')
axes[0, 0].axis('off')

# Blue 通道
axes[0, 1].imshow(b, cmap='Blues')
axes[0, 1].set_title('Blue Channel')
axes[0, 1].axis('off')

# Green 通道
axes[1, 0].imshow(g, cmap='Greens')
axes[1, 0].set_title('Green Channel')
axes[1, 0].axis('off')

# Red 通道
axes[1, 1].imshow(r, cmap='Reds')
axes[1, 1].set_title('Red Channel')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

print(f"Blue channel range: [{b.min()}, {b.max()}]")
print(f"Green channel range: [{g.min()}, {g.max()}]")
print(f"Red channel range: [{r.min()}, {r.max()}]")

In [ ]:
# 通道合併 - 創建特殊效果
# 交換通道順序
img_bgr_swapped = cv2.merge([r, g, b])  # RGM順序
img_gbr = cv2.merge([g, b, r])  # GBR順序

# 顯示
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB))
axes[0].set_title('Original (BGR)')
axes[0].axis('off')

axes[1].imshow(cv2.cvtColor(img_bgr_swapped, cv2.COLOR_BGR2RGB))
axes[1].set_title('RGB Order (Swapped)')
axes[1].axis('off')

axes[2].imshow(cv2.cvtColor(img_gbr, cv2.COLOR_BGR2RGB))
axes[2].set_title('GBR Order')
axes[2].axis('off')

plt.tight_layout()
plt.show()

## 3. HSV 色彩空間

### 為什麼使用 HSV？

HSV 色彩空間更接近人類對顏色的感知方式：
- **色相 (Hue)**: 顏色的種類（紅、綠、藍等）
- **飽和度 (Saturation)**: 顏色的純度
- **明度 (Value)**: 顏色的亮度

### HSV 的優勢
1. 色彩檢測更容易（只需指定 H 範圍）
2. 對光照變化更魯棒
3. 適合物體追蹤和色彩分割

In [ ]:
# BGR to HSV 轉換
img_hsv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HSV)

# 分離 HSV 通道
h, s, v = cv2.split(img_hsv)

# 顯示各通道
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

axes[0, 0].imshow(cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB))
axes[0, 0].set_title('Original Image')
axes[0, 0].axis('off')

axes[0, 1].imshow(h, cmap='hsv')
axes[0, 1].set_title('Hue (色相) 0-180')
axes[0, 1].axis('off')

axes[1, 0].imshow(s, cmap='gray')
axes[1, 0].set_title('Saturation (飽和度) 0-255')
axes[1, 0].axis('off')

axes[1, 1].imshow(v, cmap='gray')
axes[1, 1].set_title('Value (明度) 0-255')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

print(f"Hue range: [{h.min()}, {h.max()}]")
print(f"Saturation range: [{s.min()}, {s.max()}]")
print(f"Value range: [{v.min()}, {v.max()}]")

### HSV 色彩檢測實例

使用 HSV 檢測特定顏色（例如紅色、藍色、綠色）

In [ ]:
# 創建彩色測試圖像
test_img = np.zeros((300, 600, 3), dtype=np.uint8)

# 繪製不同顏色的矩形
cv2.rectangle(test_img, (0, 0), (200, 300), (0, 0, 255), -1)      # Red
cv2.rectangle(test_img, (200, 0), (400, 300), (0, 255, 0), -1)    # Green
cv2.rectangle(test_img, (400, 0), (600, 300), (255, 0, 0), -1)    # Blue

# 轉換到 HSV
test_hsv = cv2.cvtColor(test_img, cv2.COLOR_BGR2HSV)

# 定義顏色範圍（HSV）
# 紅色在 HSV 中跨越 0 和 180，需要兩個範圍
lower_red1 = np.array([0, 100, 100])
upper_red1 = np.array([10, 255, 255])
lower_red2 = np.array([170, 100, 100])
upper_red2 = np.array([180, 255, 255])

# 綠色範圍
lower_green = np.array([40, 100, 100])
upper_green = np.array([80, 255, 255])

# 藍色範圍
lower_blue = np.array([100, 100, 100])
upper_blue = np.array([130, 255, 255])

# 創建遮罩
mask_red1 = cv2.inRange(test_hsv, lower_red1, upper_red1)
mask_red2 = cv2.inRange(test_hsv, lower_red2, upper_red2)
mask_red = mask_red1 | mask_red2
mask_green = cv2.inRange(test_hsv, lower_green, upper_green)
mask_blue = cv2.inRange(test_hsv, lower_blue, upper_blue)

# 應用遮罩
result_red = cv2.bitwise_and(test_img, test_img, mask=mask_red)
result_green = cv2.bitwise_and(test_img, test_img, mask=mask_green)
result_blue = cv2.bitwise_and(test_img, test_img, mask=mask_blue)

# 顯示結果
fig, axes = plt.subplots(2, 4, figsize=(16, 8))

axes[0, 0].imshow(cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB))
axes[0, 0].set_title('Original')
axes[0, 0].axis('off')

axes[0, 1].imshow(mask_red, cmap='gray')
axes[0, 1].set_title('Red Mask')
axes[0, 1].axis('off')

axes[0, 2].imshow(mask_green, cmap='gray')
axes[0, 2].set_title('Green Mask')
axes[0, 2].axis('off')

axes[0, 3].imshow(mask_blue, cmap='gray')
axes[0, 3].set_title('Blue Mask')
axes[0, 3].axis('off')

axes[1, 0].axis('off')

axes[1, 1].imshow(cv2.cvtColor(result_red, cv2.COLOR_BGR2RGB))
axes[1, 1].set_title('Detected Red')
axes[1, 1].axis('off')

axes[1, 2].imshow(cv2.cvtColor(result_green, cv2.COLOR_BGR2RGB))
axes[1, 2].set_title('Detected Green')
axes[1, 2].axis('off')

axes[1, 3].imshow(cv2.cvtColor(result_blue, cv2.COLOR_BGR2RGB))
axes[1, 3].set_title('Detected Blue')
axes[1, 3].axis('off')

plt.tight_layout()
plt.show()

## 4. 灰階轉換

### 灰階轉換方法

1. **cv2.cvtColor()** - 加權平均法（推薦）
   - Gray = 0.299*R + 0.587*G + 0.114*B
   - 符合人眼感知

2. **平均法** - 簡單平均
   - Gray = (R + G + B) / 3

3. **單通道** - 直接取一個通道

In [ ]:
# 不同灰階轉換方法比較
# 方法1: cv2.cvtColor (加權平均)
gray_weighted = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)

# 方法2: 簡單平均
gray_average = np.mean(img_bgr, axis=2).astype(np.uint8)

# 方法3: 單通道（藍色）
gray_blue = img_bgr[:, :, 0]

# 方法4: 單通道（綠色）
gray_green = img_bgr[:, :, 1]

# 方法5: 單通道（紅色）
gray_red = img_bgr[:, :, 2]

# 顯示比較
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

axes[0, 0].imshow(cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB))
axes[0, 0].set_title('Original')
axes[0, 0].axis('off')

axes[0, 1].imshow(gray_weighted, cmap='gray')
axes[0, 1].set_title('Weighted Average (Recommended)')
axes[0, 1].axis('off')

axes[0, 2].imshow(gray_average, cmap='gray')
axes[0, 2].set_title('Simple Average')
axes[0, 2].axis('off')

axes[1, 0].imshow(gray_blue, cmap='gray')
axes[1, 0].set_title('Blue Channel')
axes[1, 0].axis('off')

axes[1, 1].imshow(gray_green, cmap='gray')
axes[1, 1].set_title('Green Channel')
axes[1, 1].axis('off')

axes[1, 2].imshow(gray_red, cmap='gray')
axes[1, 2].set_title('Red Channel')
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

print("Grayscale conversion comparison:")
print(f"Weighted: mean={gray_weighted.mean():.2f}, std={gray_weighted.std():.2f}")
print(f"Average: mean={gray_average.mean():.2f}, std={gray_average.std():.2f}")

## 5. LAB 色彩空間

### LAB 的特點
- **感知均勻**: 兩個顏色的距離與人眼感知的差異成正比
- **設備無關**: 不依賴於特定設備
- **色差計算**: 適合計算顏色差異

### 應用場景
1. 色彩校正
2. 色彩匹配
3. 白平衡調整
4. 膚色檢測

In [ ]:
# BGR to LAB 轉換
img_lab = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2LAB)

# 分離 LAB 通道
l, a, b_channel = cv2.split(img_lab)

# 顯示各通道
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

axes[0, 0].imshow(cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB))
axes[0, 0].set_title('Original Image')
axes[0, 0].axis('off')

axes[0, 1].imshow(l, cmap='gray')
axes[0, 1].set_title('L Channel (Lightness)')
axes[0, 1].axis('off')

axes[1, 0].imshow(a, cmap='RdYlGn_r')
axes[1, 0].set_title('a Channel (Green-Red)')
axes[1, 0].axis('off')

axes[1, 1].imshow(b_channel, cmap='YlGnBu_r')
axes[1, 1].set_title('b Channel (Blue-Yellow)')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

print(f"L channel range: [{l.min()}, {l.max()}]")
print(f"a channel range: [{a.min()}, {a.max()}]")
print(f"b channel range: [{b_channel.min()}, {b_channel.max()}]")

### LAB 應用：色彩增強

In [ ]:
# LAB 色彩增強
# 僅調整 L 通道（亮度），保持色彩
l_enhanced = cv2.equalizeHist(l)

# 合併通道
img_lab_enhanced = cv2.merge([l_enhanced, a, b_channel])

# 轉回 BGR
img_enhanced = cv2.cvtColor(img_lab_enhanced, cv2.COLOR_LAB2BGR)

# 顯示比較
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

axes[0].imshow(cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB))
axes[0].set_title('Original')
axes[0].axis('off')

axes[1].imshow(cv2.cvtColor(img_enhanced, cv2.COLOR_BGR2RGB))
axes[1].set_title('LAB Enhanced (L channel equalized)')
axes[1].axis('off')

plt.tight_layout()
plt.show()

## 6. 色彩空間轉換總覽

### OpenCV 支援的色彩空間轉換

In [ ]:
# 列出所有可用的色彩空間轉換
color_conversions = [attr for attr in dir(cv2) if attr.startswith('COLOR_')]

print(f"Total color space conversions available: {len(color_conversions)}\n")
print("Common conversions:")

common = [
    'COLOR_BGR2RGB', 'COLOR_RGB2BGR',
    'COLOR_BGR2GRAY', 'COLOR_GRAY2BGR',
    'COLOR_BGR2HSV', 'COLOR_HSV2BGR',
    'COLOR_BGR2LAB', 'COLOR_LAB2BGR',
    'COLOR_BGR2YCrCb', 'COLOR_YCrCb2BGR',
    'COLOR_BGR2HLS', 'COLOR_HLS2BGR'
]

for i, conv in enumerate(common, 1):
    print(f"{i:2d}. cv2.{conv}")

### 色彩空間轉換範例：一圖多顯

In [ ]:
# 將圖像轉換到多個色彩空間
img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
img_hsv_display = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HSV)
img_lab_display = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2LAB)
img_ycrcb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2YCrCb)
img_hls = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HLS)

# 顯示所有色彩空間
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

axes[0, 0].imshow(img_rgb)
axes[0, 0].set_title('RGB')
axes[0, 0].axis('off')

axes[0, 1].imshow(img_gray, cmap='gray')
axes[0, 1].set_title('Grayscale')
axes[0, 1].axis('off')

# 注意：HSV, LAB 等需要轉回 RGB 才能正確顯示
axes[0, 2].imshow(cv2.cvtColor(img_hsv_display, cv2.COLOR_HSV2RGB))
axes[0, 2].set_title('HSV')
axes[0, 2].axis('off')

axes[1, 0].imshow(cv2.cvtColor(img_lab_display, cv2.COLOR_LAB2RGB))
axes[1, 0].set_title('LAB')
axes[1, 0].axis('off')

axes[1, 1].imshow(cv2.cvtColor(img_ycrcb, cv2.COLOR_YCrCb2RGB))
axes[1, 1].set_title('YCrCb')
axes[1, 1].axis('off')

axes[1, 2].imshow(cv2.cvtColor(img_hls, cv2.COLOR_HLS2RGB))
axes[1, 2].set_title('HLS')
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

## 7. 實戰應用：皮膚檢測

使用多個色彩空間組合檢測皮膚區域

In [ ]:
# HSV 皮膚檢測
img_hsv_skin = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HSV)

# 定義皮膚色範圍（HSV）
lower_skin_hsv = np.array([0, 20, 70], dtype=np.uint8)
upper_skin_hsv = np.array([20, 255, 255], dtype=np.uint8)
mask_hsv = cv2.inRange(img_hsv_skin, lower_skin_hsv, upper_skin_hsv)

# YCrCb 皮膚檢測
img_ycrcb_skin = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2YCrCb)
lower_skin_ycrcb = np.array([0, 135, 85], dtype=np.uint8)
upper_skin_ycrcb = np.array([255, 180, 135], dtype=np.uint8)
mask_ycrcb = cv2.inRange(img_ycrcb_skin, lower_skin_ycrcb, upper_skin_ycrcb)

# 結合兩個遮罩
mask_combined = cv2.bitwise_and(mask_hsv, mask_ycrcb)

# 形態學操作去除噪點
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
mask_combined = cv2.morphologyEx(mask_combined, cv2.MORPH_OPEN, kernel)
mask_combined = cv2.morphologyEx(mask_combined, cv2.MORPH_CLOSE, kernel)

# 應用遮罩
result = cv2.bitwise_and(img_bgr, img_bgr, mask=mask_combined)

# 顯示
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

axes[0, 0].imshow(cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB))
axes[0, 0].set_title('Original')
axes[0, 0].axis('off')

axes[0, 1].imshow(mask_hsv, cmap='gray')
axes[0, 1].set_title('HSV Skin Mask')
axes[0, 1].axis('off')

axes[1, 0].imshow(mask_ycrcb, cmap='gray')
axes[1, 0].set_title('YCrCb Skin Mask')
axes[1, 0].axis('off')

axes[1, 1].imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
axes[1, 1].set_title('Combined Skin Detection')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

## 8. 色彩空間選擇指南

### 應用場景對照表

| 應用場景 | 推薦色彩空間 | 理由 |
|---------|------------|------|
| 色彩檢測 | HSV | 色相獨立，易於設定閾值 |
| 邊緣檢測 | Grayscale | 計算效率高 |
| 色彩校正 | LAB | 感知均勻，色差計算準確 |
| 人臉檢測 | YCrCb | 膚色檢測效果好 |
| 亮度調整 | LAB | L通道獨立調整 |
| 一般處理 | BGR/RGB | OpenCV 原生支援 |
| 深度學習 | RGB | 多數預訓練模型使用 |

### 轉換效率比較

In [ ]:
import time

# 效能測試
iterations = 100

conversions = {
    'BGR to RGB': (cv2.COLOR_BGR2RGB, cv2.COLOR_RGB2BGR),
    'BGR to GRAY': (cv2.COLOR_BGR2GRAY, None),
    'BGR to HSV': (cv2.COLOR_BGR2HSV, cv2.COLOR_HSV2BGR),
    'BGR to LAB': (cv2.COLOR_BGR2LAB, cv2.COLOR_LAB2BGR),
    'BGR to YCrCb': (cv2.COLOR_BGR2YCrCb, cv2.COLOR_YCrCb2BGR),
}

print(f"Performance test ({iterations} iterations):")
print("-" * 50)

for name, (forward, backward) in conversions.items():
    start = time.time()
    for _ in range(iterations):
        converted = cv2.cvtColor(img_bgr, forward)
        if backward:
            _ = cv2.cvtColor(converted, backward)
    elapsed = time.time() - start
    
    print(f"{name:20s}: {elapsed*1000:6.2f} ms ({elapsed*1000/iterations:.3f} ms/iter)")

## 9. 練習題

### 練習 1: 色彩空間視覺化
創建一個函數，輸入一張圖像，輸出所有色彩空間的視覺化結果。

In [ ]:
def visualize_all_color_spaces(img_bgr):
    """
    Visualize image in multiple color spaces
    
    Parameters:
    -----------
    img_bgr : numpy.ndarray
        Input image in BGR format
    """
    # TODO: 實作此函數
    # 提示:
    # 1. 轉換到 RGB, HSV, LAB, YCrCb, Grayscale
    # 2. 使用 matplotlib 創建子圖
    # 3. 分別顯示各個色彩空間
    pass

# 測試你的函數
# visualize_all_color_spaces(img_bgr)

### 練習 2: 色彩追蹤
實作一個色彩追蹤器，可以追蹤視訊中的特定顏色物體。

In [ ]:
def track_color(frame, lower_hsv, upper_hsv):
    """
    Track specific color in a frame
    
    Parameters:
    -----------
    frame : numpy.ndarray
        Input frame in BGR format
    lower_hsv : numpy.ndarray
        Lower HSV threshold
    upper_hsv : numpy.ndarray
        Upper HSV threshold
        
    Returns:
    --------
    mask : numpy.ndarray
        Binary mask of detected color
    result : numpy.ndarray
        Frame with detected color highlighted
    """
    # TODO: 實作色彩追蹤
    # 提示:
    # 1. 轉換到 HSV
    # 2. 使用 inRange 創建遮罩
    # 3. 形態學操作去噪
    # 4. 找到輪廓並繪製
    pass

### 練習 3: 自動白平衡
實作自動白平衡算法，使用 LAB 色彩空間。

In [ ]:
def auto_white_balance(img_bgr):
    """
    Automatic white balance using LAB color space
    
    Parameters:
    -----------
    img_bgr : numpy.ndarray
        Input image in BGR format
        
    Returns:
    --------
    balanced : numpy.ndarray
        White balanced image
    """
    # TODO: 實作自動白平衡
    # 提示:
    # 1. 轉換到 LAB
    # 2. 計算 a, b 通道的平均值
    # 3. 調整 a, b 通道使其接近灰色
    # 4. 轉回 BGR
    pass

## 10. 總結

### 關鍵要點

1. **色彩空間多樣性**: 不同色彩空間適用於不同應用
2. **HSV 的實用性**: 色彩檢測的最佳選擇
3. **LAB 的準確性**: 色彩測量和校正的標準
4. **轉換效率**: 考慮效能時選擇合適的色彩空間
5. **組合使用**: 多個色彩空間組合可獲得更好效果

### 延伸學習

- YUV 色彩空間在視訊編碼中的應用
- CIE XYZ 標準色彩空間
- 色域與色彩管理
- 深度學習中的色彩增強技術

### 參考資源

- OpenCV Color Space Conversions: https://docs.opencv.org/4.x/de/d25/imgproc_color_conversions.html
- HSV Color Space: https://en.wikipedia.org/wiki/HSL_and_HSV
- CIELAB Color Space: https://en.wikipedia.org/wiki/CIELAB_color_space

---

## 下一步

完成本模組後，建議繼續學習：
- **2.2.4 算術運算** - 圖像混合與合成
- **3.1.1 濾波與平滑** - 圖像降噪技術
- **3.2.1 直方圖處理** - 圖像增強方法

**練習建議**: 嘗試使用不同色彩空間解決實際問題，如物體追蹤、色彩校正等。